In [ ]:
import os
import schedule
import datetime
import calendar
from time import sleep
from zipfile import ZipFile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

def scheduleScrapper():
    def joinYMD(string):
        year = string.split('-')[0]
        month = string.split('-')[1]
        day = string.split('-')[2]
        return str(year) + '/' + str(month) + '/' + str(day)

    # Convert scrapDataDate to save as zip file name
    def joinYMDZipFile(string):
        year = string.split('-')[0]
        month = string.split('-')[1]
        day = string.split('-')[2]
        return str(year) + '_' + str(month) + '_' + str(day)

    # Get yesterday
    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=1)

    # Transform the format of the scrap date
    scrapDataDate = joinYMD(str(yesterday))

    class color:
        SUCCESS = '\033[92m'
        WARNING = '\033[93m'
        FAIL = '\033[91m'

    # Function to get the correct xpath of each calendar
    def getCurrentCalendarXPATH(calendar_id, year, month):
        return "//table[@id='" + calendar_id + "_calendar_start_" + year + '_' + month + "']//a"

    print('Executing: Prepare Web Driver (Chrome) and Login...')
    sleep(5)
    # Initialize Chrome
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.implicitly_wait(30)

    # Navigate to D1D system
    url = 'http://itpv2.transtron.fujitsu.com/'
    driver.get(url)

    # Fill login information
    sleep(1)
    driver.find_element(By.ID, 'userid').send_keys('izmb-free')
    driver.find_element(By.ID, 'password').send_keys('izumi001')

    # Login
    sleep(1)
    driver.find_element(By.ID, 'login').click()

    # Waiting the client loading
    sleep(60)

    # Reset action
    def resetActionKey():
        resetSpace = driver.find_element(
            By.XPATH, '//*[@id="F04_mainview"]/div[2]/div[1]')
        resetSpace.click()

    print('Executing: Alcohol Measurement Result...')
    sleep(5)

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Daily Report
    sleep(1)
    daily_report = driver.find_element(By.ID, 'F20')
    ActionChains(driver).move_to_element(daily_report).perform()

    # Navigate to Alcohol Measurement Result
    sleep(1)
    driver.find_element(By.XPATH, '//*[@id="F25"]').click()

    # Select department
    sleep(1)
    select_department = driver.find_element(By.ID, 'F25eigyousyo_list')
    select_department.click()
    sleep(1)
    driver.find_element(
        By.XPATH, '//*[@id="F25eigyousyo_list"]/option[2]').click()

    # Handle the scrapper date - should be yester day of today
    # Start date
    sleep(10)
    select_department.click()
    sleep(1)
    start_date = driver.find_element(By.ID, 'F25_StartDate')
    start_date.click()

    index = 0

    # When focus in the input of rhe calendar, the cursor is placed at the last 2 to the right
    # need to move 2 steps further to move the cursor to the max right
    sleep(1)
    while index <= 2:
        start_date.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        start_date.send_keys(Keys.BACKSPACE)
        index += 1

    start_date.send_keys(scrapDataDate)

    sleep(1)
    resetActionKey()

    # End date
    end_date = driver.find_element(By.ID, 'F25_EndDate')
    end_date.click()

    index = 0

    while index <= 2:
        end_date.send_keys(Keys.ARROW_RIGHT)
        index += 1

    while index <= 12:
        end_date.send_keys(Keys.BACKSPACE)
        index += 1

    end_date.send_keys(scrapDataDate)

    sleep(1)
    resetActionKey()

    # Options in the select department
    sleep(1)
    options = [item for item in select_department.find_elements(
        By.TAG_NAME, 'option')]
    del options[0]

    count = 0
    existMessageBox = True
    listDepartmentHaveData = []

    for element in (options):
        print(f"{color.WARNING}Processing department: " + str(element.text))
        element.click()

        # Check if the selected department have data or not
        try:
            noDataNotifyModal = driver.find_element(
                By.ID, 'function_window_F25_messagebox_0')
        except NoSuchElementException:
            existMessageBox = False

        # If the selected departsment have data then download the data
        # otherwise, click on the OK button to turn off the modal
        if existMessageBox == True:
            driver.find_element(By.ID, 'msg_confirm').click()
        elif existMessageBox == False:
            driver.find_element(By.ID, 'P25_csvout_button').click()
            listDepartmentHaveData.append(element.text)

        if existMessageBox == True:
            print(f"{color.FAIL}Department: " +
                  element.text + " have no data, passed !")
        else:
            print(f"{color.SUCCESS}Department: " +
                  element.text + " proceed successfully !")

        print('--------------------------------------------------------------')
        existMessageBox = True
        sleep(2)
        count += 1

    directory = 'C:/Users/HaThaiViet/Downloads'

    listFiles = []

    for filename in os.listdir(directory):
        if filename.endswith('.zip') and filename.startswith('アルコール測定結果'):
            listFiles.append(os.path.join(directory, filename))
            continue
        else:
            continue

    # Close the alcohol measurement result scrapper
    index = 30
    while 0 <= index:
        print("Continue Daily Results Scrapper in: " + str(index))
        sleep(1)
        index -= 1

    print('Executing: Other Results...')
    sleep(5)

    # Open Menu
    sleep(1)
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    ActionChains(driver).move_to_element(function_list).perform()

    # Open Results Dropdown
    sleep(1)
    driver.find_element(By.ID, 'F40_dl').click()

    # Vehicel Expense Report
    sleep(1)
    vehicle_expense_report = driver.find_element(By.XPATH, '//*[@id="F45"]')
    vehicle_expense_report.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F45_eigyousyoLevel_view"]/li/div/span/div/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F45_StartDate")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F45_EndDate")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F45_CSV_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Vehicle Expense Report -----------------------

    # ----------------------- Start Driver Expense Report -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Driver Expense Report
    sleep(1)
    driver_expense_report = driver.find_element(By.XPATH, '//*[@id="F46"]')
    driver_expense_report.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F46_eigyousyoLevel_view"]/li/div/span/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F46_from")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F46_to")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F46_csvout_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Driver Expense Report -----------------------

    # ----------------------- Start Driving Report by Driver -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Driving Report by Driver
    sleep(1)
    driving_report_by_driver = driver.find_element(By.XPATH, '//*[@id="F42"]')
    driving_report_by_driver.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F42_eigyousyoLevel_view"]/li/div/span/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F42_from")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F42_to")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F42_csvout_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Driving Report by Driver -----------------------

    # ----------------------- Start Driver Report by Vehicle -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Driver Report by Vehicle
    sleep(1)
    driver_report_by_vehicle = driver.find_element(By.XPATH, '//*[@id="F43"]')
    driver_report_by_vehicle.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F43_eigyousyoLevel_view"]/li/div/span/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F43_StartDate")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F43_EndDate")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F43_CSV_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Driver Report by Vehicle -----------------------

    # ----------------------- Start Yearly Mileage Report -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Yearly Mileage Report
    sleep(1)
    yearly_mileage_report = driver.find_element(By.XPATH, '//*[@id="F44"]')
    yearly_mileage_report.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F44_eigyousyoLevel_view"]/li/div/span/div').click()

    sleep(1)

    # Open dropdown option Trip Start Year
    trip_start_year = driver.find_element(By.ID, 'ComboBox_StartYear')
    trip_start_year.click()

    # Set the year to current year
    sleep(1)
    select_trip_start_year = Select(trip_start_year)
    select_trip_start_year.select_by_visible_text(str(today.year))

    # Open dropdown option Trip Start Month
    trip_start_month = driver.find_element(By.ID, 'ComboBox_StartMonth')
    trip_start_month.click()

    # Set the year to current year
    sleep(1)
    select_trip_start_month = Select(trip_start_month)
    select_trip_start_month.select_by_visible_text(str(today.month))

    # Open dropdown option Trip End Year
    trip_end_year = driver.find_element(By.ID, 'ComboBox_EndYear')
    trip_end_year.click()

    # Set the year to current year
    sleep(1)
    select_trip_end_year = Select(trip_end_year)
    select_trip_end_year.select_by_visible_text(str(today.year))

    # Open dropdown option Trip End Month
    trip_end_month = driver.find_element(By.ID, 'ComboBox_EndMonth')
    trip_end_month.click()

    # Set the year to current year
    sleep(1)
    select_trip_end_month = Select(trip_end_month)
    select_trip_end_month.select_by_visible_text(str(today.month))

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F33_CSV_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Yearly Mileage Report -----------------------

    # ----------------------- Start Digital Tachograph Data -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Digital Tachograph Data
    sleep(1)
    digital_tachograph_data = driver.find_element(By.XPATH, '//*[@id="F49"]')
    digital_tachograph_data.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F49_eigyousyoLevel_view"]/li/div/span/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F49_from")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F49_get_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Digital Tachograph Data -----------------------

    # ----------------------- Start Safety Driving Ranking Report -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Safery Driving Ranking Report
    sleep(1)
    safety_driving_ranking_report = driver.find_element(
        By.XPATH, '//*[@id="F41"]')
    safety_driving_ranking_report.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F41_eigyousyoLevel_view"]/li/div/span/div').click()

    # Report check button
    sleep(5)
    report_check = driver.find_element(
        By.XPATH, '//*[@id="F41_main_form"]/p[2]/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F41_StartDate")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F41_EndDate")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F41_CSV_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Safety Driving Ranking Report -----------------------

    # ----------------------- Start Driver Report -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Driver Report
    sleep(1)
    driver_report = driver.find_element(By.XPATH, '//*[@id="F4B"]')
    driver_report.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F41_eigyousyoLevel_view"]/li/div/span/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F41_StartDate")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F41_EndDate")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F41_CSV_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Driver Report -----------------------

    # ----------------------- Start Driving Report by Vehicle -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Driving Report By Vehicle
    sleep(1)
    driving_report_by_vehicle = driver.find_element(By.XPATH, '//*[@id="F4C"]')
    driving_report_by_vehicle.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F41_eigyousyoLevel_view"]/li/div/span/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F41_StartDate")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F41_EndDate")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F41_CSV_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End Driving Report by Vehicle -----------------------

    # ----------------------- Start 拠点別集計表 -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # 拠点別集計表
    sleep(1)
    拠点別集計表 = driver.find_element(By.XPATH, '//*[@id="P21"]')
    拠点別集計表.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="P21_eigyousyoLevel_view"]/li/div/span/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "P21_StartDate")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "P21_EndDate")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'P21_CSV_button').click()

    # Waiting for the handling and dowload process
    sleep(60)

    # ----------------------- End 拠点別集計表 -----------------------

    # ----------------------- Start 車両別安全評価実績表 -----------------------

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # 車両別安全評価実績表
    sleep(1)
    車両別安全評価実績表 = driver.find_element(By.XPATH, '//*[@id="F4G"]')
    車両別安全評価実績表.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(
        By.XPATH, '//*[@id="F4G_eigyousyoLevel_view"]/li/div/span/div').click()

    # Remove 営業所毎にページを分けて印刷する checkbox
    sleep(5)
    last_checkbox = driver.find_element(
        By.XPATH, '//*[@id="F4G_main_form"]/p[3]/div').click()

    # Check 車両別安全評価実績表 （積算）
    sleep(5)
    second_checkbox = driver.find_element(
        By.XPATH, '//*[@id="F4G_main_form"]/p[2]/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F4G_StartDate")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F4G_EndDate")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F4G_CSV_button').click()

    sleep(60)
    print('Start Assistance Data Output')

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Open Assistance Dropdown
    sleep(1)
    assistance = driver.find_element(By.ID, 'F80_dl')
    assistance.click()

    sleep(1)
    data_output = driver.find_element(By.ID, 'F81').click()

    print('------------------------ Start Registration Files ------------------------')

    # Options in the select department
    # Select department
    sleep(1)
    select_department = driver.find_element(By.ID, 'F81eigyousyo_list')
    select_department.click()

    sleep(1)
    options = [item for item in select_department.find_elements(
        By.TAG_NAME, 'option')]

    count = 0
    existMessageBox = True

    # Check all the checkbox
    branch_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[1]/table/tbody/tr[1]/td[1]/div')
    branch_data.click()

    shipper_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[1]/table/tbody/tr[1]/td[2]/div')
    shipper_data.click()

    driver_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[1]/table/tbody/tr[1]/td[3]/div')
    driver_data.click()

    point_information = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[1]/table/tbody/tr[1]/td[4]/div')
    point_information.click()

    vehicle_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[1]/table/tbody/tr[2]/td[1]/div')
    vehicle_data.click()

    goods_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[1]/table/tbody/tr[2]/td[2]/div')
    goods_data.click()

    group_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[1]/table/tbody/tr[2]/td[3]/div')
    group_data.click()

    for element in (options):
        print(f"{color.WARNING}Processing department: " + str(element.text))
        element.click()

        # Handle the scrapper date - should be yester day of today
        # Start date
        sleep(1)
        start_date = driver.find_element(By.ID, 'StartDate')
        start_date.click()

        index = 0

        # When focus in the input of rhe calendar, the cursor is placed at the last 2 to the right
        # need to move 2 steps further to move the cursor to the max right
        sleep(1)
        while index <= 2:
            start_date.send_keys(Keys.ARROW_RIGHT)
            index += 1

        sleep(1)
        while index <= 12:
            start_date.send_keys(Keys.BACKSPACE)
            index += 1

        start_date.send_keys(scrapDataDate)

        sleep(1)
        resetActionKey()

        # End date
        end_date = driver.find_element(By.ID, 'EndDate')
        end_date.click()

        index = 0

        while index <= 2:
            end_date.send_keys(Keys.ARROW_RIGHT)
            index += 1

        while index <= 12:
            end_date.send_keys(Keys.BACKSPACE)
            index += 1

        end_date.send_keys(scrapDataDate)

        sleep(1)
        resetActionKey()

        driver.find_element(By.ID, 'F81_csvout_button').click()

        try:
            noDataNotifyModal = driver.find_element(
                By.ID, 'function_window_F81_messagebox_0')
        except NoSuchElementException:
            existMessageBox = False

        if existMessageBox == True:
            driver.find_element(By.ID, 'msg_confirm').click()

        print('--------------------------------------------------------------')
        count += 1

    print('------------------------ End Registration Files ------------------------')

    print('------------------------ Start Counting File ------------------------')

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    sleep(1)
    data_output = driver.find_element(By.ID, 'F81').click()
    
    listCountingFileOption = []
    temperature_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[1]/td[1]/div'
    )

    speed_frequency_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[1]/td[2]/div'
    )

    fee_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[1]/td[3]/div'
    )

    fee_data_sub = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[2]/td[3]/div'
    )

    engine_rotation_frequency_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[1]/td[4]/div'
    )

    work_results_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[3]/td[1]/div'
    )

    acc_dec_frequency_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[3]/td[2]/div'
    )

    daily_report_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[3]/td[3]/div'
    )

    daily_report_data_sub = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[4]/td[3]/div'
    )

    gear_rate_frequency_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[3]/td[4]/div'
    )

    sleepiness_detection_data = driver.find_element(
        By.XPATH, '//*[@id="data_out_form"]/section/article[2]/table/tbody/tr[5]/td[3]/div'
    )

    movie_upload_history_data = driver.find_element(
        By.XPATH, '//*[@id="F81_VideoGetData"]/div'
    )

    dsm_data = driver.find_element(
        By.ID, 'F81_DMSRadio'
    )

    mobileye_data = driver.find_element(
        By.ID, 'F81_MobileyeRadio'
    )

    listCountingFileOption = [
        temperature_data,
        speed_frequency_data,
        fee_data,
        fee_data_sub,
        engine_rotation_frequency_data,
        work_results_data,
        acc_dec_frequency_data,
        daily_report_data,
        daily_report_data_sub,
        gear_rate_frequency_data,
        sleepiness_detection_data,
        movie_upload_history_data,
        dsm_data,
        mobileye_data
    ]

    # Options in the select department
    # Select department
    sleep(1)
    select_department = driver.find_element(By.ID, 'F81eigyousyo_list')
    select_department.click()

    sleep(1)
    options = [item for item in select_department.find_elements(
        By.TAG_NAME, 'option')]

    count = 0
    existMessageBox = True

    for element in (options):
        print(f"{color.WARNING}Processing department: " + str(element.text))
        element.click()

        # Handle the scrapper date - should be yester day of today
        # Start date
        sleep(1)
        start_date = driver.find_element(By.ID, 'StartDate')
        start_date.click()

        index = 0

        # When focus in the input of rhe calendar, the cursor is placed at the last 2 to the right
        # need to move 2 steps further to move the cursor to the max right
        sleep(1)
        while index <= 2:
            start_date.send_keys(Keys.ARROW_RIGHT)
            index += 1

        sleep(1)
        while index <= 12:
            start_date.send_keys(Keys.BACKSPACE)
            index += 1

        start_date.send_keys(scrapDataDate)

        sleep(1)
        resetActionKey()

        # End date
        end_date = driver.find_element(By.ID, 'EndDate')
        end_date.click()

        index = 0

        while index <= 2:
            end_date.send_keys(Keys.ARROW_RIGHT)
            index += 1

        while index <= 12:
            end_date.send_keys(Keys.BACKSPACE)
            index += 1

        end_date.send_keys(scrapDataDate)

        sleep(1)
        resetActionKey()

        index = 0
        while index < len(listCountingFileOption):
            sleep(2)

            listCountingFileOption[index].click()
            
            if index <= 9:
                print('| ' + listCountingFileOption[index].text + ' |')
            elif index == 10:
                print('| DSM Data |')
            elif index == 11:
                print('| Mobileye Data |')
            
            if index == 2:
                listCountingFileOption[3].click()
            elif index == 6:
                listCountingFileOption[7].click()
            
            sleep(10)
            driver.find_element(By.ID, 'F81_csvout_button').click()
            sleep(5)

            try:
                noDataNotifyModal = driver.find_element(
                    By.ID, 'function_window_F81_messagebox_0')
            except NoSuchElementException:
                existMessageBox = False

            if existMessageBox == True:
                driver.find_element(By.ID, 'msg_confirm').click()
                sleep(2)
        
            if index == 2:
                index += 2
            elif index == 6:
                index += 2
            else:
                index += 1

        print('--------------------------------------------------------------')
        count += 1

    print('------------------------ End Counting Files ------------------------')

    sleep(60)
    print('Start Safety Driving Ranking Report for Posting')

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Open Safety Dropdown
    sleep(1)
    safety = driver.find_element(By.ID, 'F30')
    safety.click()

    sleep(1)
    safety_driving_ranking_report_for_posting = driver.find_element(By.ID, 'F33')
    safety_driving_ranking_report_for_posting.click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    checkbox = driver.find_element(By.XPATH, '//*[@id="F33_eigyousyoLevel_view"]/li/div/span/div/div').click()

    sleep(5)
    counting_period = driver.find_element(By.XPATH, '//*[@id="F33_article-setting"]/p[3]/div')
    counting_period.click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "F33_StartDate")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "F33_EndDate")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    sleep(2)
    driver.find_element(By.ID, 'F33_CSV_button').click()

    # Waiting for the handling and dowload process
    sleep(60)
    print('Start Fuel Monitoring')

    # Open Menu
    sleep(1)
    btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
    btn_menu.click()

    # Hover Function List
    sleep(1)
    function_list = driver.find_element(By.ID, 'F100')
    hover = ActionChains(driver).move_to_element(function_list)
    hover.perform()

    # Open Results Dropdown
    sleep(1)
    fuel_monitoring = driver.find_element(By.ID, 'FE0_dl')
    fuel_monitoring.click()

    sleep(1)
    乗務員別燃費集計表 = driver.find_element(By.ID, 'FE4').click()

    # Cancel button to ensure the menu is turn off
    sleep(10)
    resetActionKey()

    # Checkbox button
    sleep(5)
    driver.find_element(By.XPATH, '//*[@id="FE4_eigyousyoLevel_view"]/li/div/span/div').click()

    # Handle the Start Date
    sleep(1)
    inputStartDate = driver.find_element(By.ID, "FE4_from")
    inputStartDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputStartDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputStartDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputStartDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Handle the End Date
    sleep(1)
    inputEndDate = driver.find_element(By.ID, "FE4_to")
    inputEndDate.click()

    index = 0

    sleep(1)
    while index <= 2:
        inputEndDate.send_keys(Keys.ARROW_RIGHT)
        index += 1

    sleep(1)
    while index <= 12:
        inputEndDate.send_keys(Keys.BACKSPACE)
        index += 1

    inputEndDate.send_keys(scrapDataDate)

    sleep(2)
    resetActionKey()

    # Dowload button
    buttonDownload = driver.find_element(By.ID, 'FE4_csvout_button').click()
    sleep(60)

    existExceptionBox = True

    try:
        exception_box = driver.find_element(By.ID, 'function_window_FE4_messagebox_0')
    except NoSuchElementException:
        existExceptionBox = False

    print(existExceptionBox)

    while existExceptionBox == True:
        driver.find_element(By.ID, 'msg_confirm').click()
        sleep(2)
        buttonDownload.click()
        sleep(60)

        try:
            exception_box = driver.find_element(By.ID, 'function_window_FE4_messagebox_0')
        except NoSuchElementException:
            existExceptionBox = False

    # Waiting for the handling and dowload process
    sleep(60)
    print('FINISHED')

# Run every day at 16:00 PM
scheduledTime = "09:05"
schedule.every().day.at(scheduledTime).do(scheduleScrapper)

while True:
    schedule.run_pending()